In [88]:
# Importamos TfidfVectorizer de scikit-learn para convertir texto en una representación numérica basada en la frecuencia de términos
from sklearn.feature_extraction.text import TfidfVectorizer
# Importamos train_test_split de scikit-learn para dividir los datos en conjuntos de entrenamiento y validación de forma aleatoria
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#Herramientas para evaluar el rendimiento del modelo.
from sklearn.metrics import classification_report, accuracy_score
from scipy.sparse import hstack
import pandas as pd

In [89]:
def entrenarAlgoritmo(file_path,*args):

    data = pd.read_csv(file_path) #data se convierte en un DataFrame que contiene todas las columnas del conjunto de datos.
    data.fillna('', inplace=True) #evitamos valores nulos

    #características individuales textuales como datos de entrada al modelo.
    features_text = data[[feature for feature in args if feature in data.columns]]

    labels = data['Binary Classification']
    
    vectorizers = {} #creamos un diccionario para almacenar los tf idf de cada columna
    X_texts = []

    for column in features_text.columns: #iteramos sobre cada features_text y creamos un tf-idf para cada columna
        vectorizer = TfidfVectorizer(max_features=100)
        X_col = vectorizer.fit_transform(features_text[column])
        vectorizers[column] = vectorizer    #almacenamos en el diccionario con el nombre de la columna
        X_texts.append(X_col) #las transformaciones X_col se almacenan en la lista X_text
    #conjunto de características propias textuales
    X_text = hstack(X_texts)

    #conjunto de características propias textuales y numéricas
    
    X = hstack([X_text])
    X_train, X_val, y_train, y_val = train_test_split(X, labels, test_size=0.2, random_state=42) #80% de los datos (test_size=0.2) para entrenar (X_train, y_train), y el 20% restante para evaluar (X_val, y_val).
    return X_train, y_train, X_val, y_val

In [90]:
import os
def reporte(y_val, y_pred, filename='report.csv', imprimir=True):
    # Evaluar el rendimiento
    report = classification_report(y_val, y_pred, output_dict=True)
    accuracy = accuracy_score(y_val, y_pred)
    
    if imprimir:
        report_df = pd.DataFrame(report).transpose()
        report_df.loc['accuracy'] = [accuracy, None, None, None]
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename, index_col=0)
            combined_df = pd.concat([existing_df,report_df])
            combined_df.to_csv(filename,index = True)
        else:
            report_df.to_csv(filename,index = True) 
        print("Reporte de clasificación:",filename,"\n", classification_report(y_val, y_pred))
    else:
        accuracy

In [91]:
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.exceptions import UndefinedMetricWarning

def SVM(X_train, y_train, X_val, y_val, kernel='linear', C=3, Show_report=None,bandera=''):
    """
    Parámetros:
    - kernel: Tipo de kernel a usar en el SVM (por defecto 'linear').'linear', 'rbf', 'precomputed', 'sigmoid', 'poly'
    - C: Parámetro de regularización (por defecto 3).
    """
    # Definir el modelo SVM
    model = SVC(kernel=kernel, C=C)
    
    # Entrenar el modelo y hacer las predicciones en el conjunto de validación
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    # Generar reporte
    #reporte(y_val, y_pred, 'SVM_reporte'+bandera+'.csv',Show_report)
    reporte(y_val, y_pred, 'reporte_SVM.csv',Show_report)
    return accuracy_score(y_val, y_pred)

def regression(X_train, y_train, X_val, max_iter=2000, solver='liblinear', Show_report=None,bandera=''):
    """
    Parámetros:
    - max_iter: Número máximo de iteraciones (por defecto 2000).
    - solver: Algoritmo a usar en el problema de optimización (por defecto 'liblinear').
        'newton-cholesky', 'newton-cg', 'liblinear', 'saga', 'sag', 'lbfgs'
    """
    # Definir el modelo de regresión logística
    model = LogisticRegression(max_iter=max_iter, solver=solver)
    try:
        # Suprimir warnings de convergencia
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            # Entrenar el modelo
            model.fit(X_train, y_train)
        
        # Hacer las predicciones en el conjunto de validación
        y_pred = model.predict(X_val)

        # Generar reporte
        reporte(y_val, y_pred, 'reporte_regression'+bandera+'.csv', Show_report)
        return accuracy_score(y_val, y_pred)
    
    except ConvergenceWarning:
        print("El modelo no ha convergido. Ajusta los parámetros o intenta con otro solver.")
        return None

def decision_tree(X_train, y_train, X_val, random_state=42, Show_report=None,bandera=''):
    """
    Parámetros:
    - random_state: Semilla para la generación de números aleatorios (por defecto 42).
    """
    # Definir el modelo de Árbol de Decisión
    model = DecisionTreeClassifier(random_state=random_state)
    
    # Entrenar el modelo y hacer las predicciones en el conjunto de validación
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    reporte(y_val, y_pred, 'reporte_trees'+bandera+'.csv', Show_report)
    return accuracy_score(y_val, y_pred)


def random_forest(X_train, y_train, X_val, n_estimators=100, random_state=42, Show_report=None,bandera=''):
    """
    Parámetros:
    - n_estimators: Número de árboles en el bosque (por defecto 100).
    - random_state: Semilla para la generación de números aleatorios (por defecto 42).
    """
    # Definir el modelo Random Forest
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    try:
        # Suprimir warnings de métricas indefinidas
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
            # Entrenar el modelo
            model.fit(X_train, y_train)
        
        # Hacer las predicciones en el conjunto de validación
        y_pred = model.predict(X_val)

        # Generar reporte
        reporte(y_val, y_pred, 'reporte_rf'+bandera+'.csv', Show_report)
        #return(classification_report(y_val, y_pred, zero_division=0))
        return accuracy_score(y_val, y_pred)
    
    except UndefinedMetricWarning:
        print("Advertencia: Métrica indefinida. Ajusta los parámetros o revisa los datos.")
        return None

In [92]:
import time
def validacion(model = None):
    inicio = time.time()

    kernels = ['linear', 'rbf', 'poly', 'sigmoid']
    best_kernel = None

    solvers = ['newton-cholesky', 'newton-cg', 'liblinear', 'saga', 'sag', 'lbfgs']
    best_solver = None

    best_value = 0

    if model == 1:
        for kernel in kernels:
            for param in range(1, 300):
                value = SVM(X_train, y_train, X_val, y_val, kernel, param, False, '')
                if value is not None and value > best_value:
                    best_value = value
                    best_param = param
                    best_kernel = kernel
        fin = time.time()
        tiempoEjec = fin - inicio
        print(f"Con {tiempoEjec} segundos en ejecución.")
        print(f"Mejor parámetro C: {best_param}")
        print(f"Mejor kernel: {best_kernel}")
        SVM(X_train, y_train, X_val, y_val, best_kernel, best_param, True, '')
    elif model == 2:
        for solver in solvers:
            for param in range(800, 1000):
                value = regression(X_train, y_train, X_val, param, solver,False,'')
                if value is not None and value > best_value:
                    best_value = value
                    best_param = param
                    best_solver = solver
        fin = time.time()
        tiempoEjec = fin - inicio
        print(f"Con {tiempoEjec} segundos en ejecución.")
        print(f"Mejor max iteraciones: {best_param}")
        print(f"Mejor solver: {best_solver}")
        regression(X_train, y_train, X_val, best_param, best_solver,True,'')
    elif model == 3:
        for rstate in range(1, 500):
            value = decision_tree(X_train, y_train, X_val, rstate, False, '')
            if value is not None and value > best_value:
                    best_value = value
                    best_random_state = rstate
        fin = time.time()
        tiempoEjec = fin - inicio
        print(f"Con {tiempoEjec} segundos en ejecución.")
        print(f"Mejor random state: {best_random_state}")
        decision_tree(X_train, y_train, X_val, best_random_state, True, '')
    elif model == 4:
        for n in range(90,110):
            for param in range(20,50):
                value = random_forest(X_train, y_train, X_val,n, param, False,'')
                if value is not None and value > best_value:
                    best_value = value
                    best_random_state = param
                    best_n_estimator = n
        fin = time.time()
        tiempoEjec = fin - inicio
        print(f"Con {tiempoEjec} segundos en ejecución.")
        print(f"Mejor random state: {best_random_state}")
        print(f"Mejor n estimator: {best_n_estimator}")
        value = random_forest(X_train, y_train, X_val,best_n_estimator, best_random_state, True,'')
    #print(f"Mejor precisión: {best_value}")

In [93]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.714640378952026 segundos en ejecución.
Mejor parámetro C: 6
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 26.843952178955078 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.1345696449279785 segundos en ejecución.
Mejor random

In [94]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology', 'Characters, plot and writing style','Summary 1','Summary 2', 'Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.8430163860321045 segundos en ejecución.
Mejor parámetro C: 6
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 26.540892362594604 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.1220743656158447 segundos en ejecución.
Mejor rando

In [95]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1', 'Future projections', 'Summary 2', 'Ideology', 'Summary 3','Characters, plot and writing style','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.91471266746521 segundos en ejecución.
Mejor parámetro C: 6
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 26.790419578552246 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.0858263969421387 segundos en ejecución.
Mejor random 

In [96]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Ideology','World view', 'Summary 3', 'Summary 2', 'Characters, plot and writing style','Future projections','Summary 1')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.893505096435547 segundos en ejecución.
Mejor parámetro C: 6
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 28.585317373275757 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.2295632362365723 segundos en ejecución.
Mejor random

In [97]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Future projections', 'Summary 2', 'Summary 3', 'Characters, plot and writing style','Summary 1','Ideology','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.992349147796631 segundos en ejecución.
Mejor parámetro C: 6
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 27.40929651260376 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.2377898693084717 segundos en ejecución.
Mejor random 

In [98]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3',  'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.051859140396118 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 30.39029622077942 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.127896547317505 segundos en ejecución.
Mejor random st

In [99]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view',  'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.042092323303223 segundos en ejecución.
Mejor parámetro C: 120
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 23.4869065284729 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.1389894485473633 segundos en ejecución.
Mejor random

In [100]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view', 'Future projections','Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.167653560638428 segundos en ejecución.
Mejor parámetro C: 46
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 33.21533989906311 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.231332540512085 segundos en ejecución.
Mejor random 

In [101]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view', 'Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.473556518554688 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 26.18705940246582 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.142171859741211 segundos en ejecución.
Mejor random st

In [102]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'Characters, plot and writing style','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.923403978347778 segundos en ejecución.
Mejor parámetro C: 8
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 29.516236782073975 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 3.1289925575256348 segundos en ejecución.
Mejor random

In [103]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'Characters, plot and writing style','Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.411099910736084 segundos en ejecución.
Mejor parámetro C: 2
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 29.491589069366455 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.20      0.50      0.29         2
Non-dystopia       0.83      0.56      0.67         9

    accuracy                           0.55        11
   macro avg       0.52      0.53      0.48        11
weighted avg       0.72      0.55      0.60        11


Con 3.134880304336548 segundos en ejecución.
Mejor random s

In [104]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'Characters, plot and writing style','Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.925650119781494 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 23.47234296798706 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.0381855964660645 segundos en ejecución.
Mejor random s

In [105]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.788722515106201 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 23.043797492980957 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 3.023928642272949 segundos en ejecución.
Mejor random s

In [106]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3', 'World view', 'Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 8.088018417358398 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 30.360719919204712 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 3.3375303745269775 segundos en ejecución.
Mejor random 

In [107]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.802532196044922 segundos en ejecución.
Mejor parámetro C: 28
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 21.733561277389526 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.25      1.00      0.40         2
Non-dystopia       1.00      0.33      0.50         9

    accuracy                           0.45        11
   macro avg       0.62      0.67      0.45        11
weighted avg       0.86      0.45      0.48        11


Con 3.491300106048584 segundos en ejecución.
Mejor random

In [108]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Summary 3', 'World view', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.724876880645752 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 29.64835810661316 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.0934207439422607 segundos en ejecución.
Mejor random s

In [109]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'World view', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.916085243225098 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 26.788063526153564 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.856867551803589 segundos en ejecución.
Mejor random s

In [110]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 3', 'World view', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.54454779624939 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 28.927855491638184 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.938899278640747 segundos en ejecución.
Mejor random st

In [111]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Summary 3', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.399561882019043 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 27.425156354904175 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.8920466899871826 segundos en ejecución.
Mejor random 

In [112]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'World view', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.021720886230469 segundos en ejecución.
Mejor parámetro C: 119
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 21.76921820640564 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.717883586883545 segundos en ejecución.
Mejor random

In [113]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 3', 'World view', 'Future projections', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.219018459320068 segundos en ejecución.
Mejor parámetro C: 68
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 25.432384490966797 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.997074604034424 segundos en ejecución.
Mejor random state

In [114]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2', 'Characters, plot and writing style','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.136366605758667 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 20.070711612701416 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.890679359436035 segundos en ejecución.
Mejor random state: 

In [115]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Summary 3', 'Characters, plot and writing style','Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.8343470096588135 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 21.74617624282837 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 3.000303268432617 segundos en ejecución.
Mejor random s

In [116]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1', 'Summary 3', 'World view', 'Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.2344818115234375 segundos en ejecución.
Mejor parámetro C: 106
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       1.00      1.00      1.00         2
Non-dystopia       1.00      1.00      1.00         9

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11


Con 23.20775032043457 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 3.0261125564575195 segundos en ejecución.
Mejor rand

In [117]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view','Summary 1','Summary 2')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.058148145675659 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 17.684473276138306 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.9792919158935547 segundos en ejecución.
Mejor random 

In [118]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Future projections','Summary 1','Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.075458526611328 segundos en ejecución.
Mejor parámetro C: 4
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 17.770318031311035 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.892254114151001 segundos en ejecución.
Mejor random s

In [119]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Ideology','Summary 2','Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.992778778076172 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 15.557100296020508 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 2.8166329860687256 segundos en ejecución.
Mejor random 

In [120]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 2')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.0932323932647705 segundos en ejecución.
Mejor parámetro C: 3
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 12.41527795791626 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.22      1.00      0.36         2
Non-dystopia       1.00      0.22      0.36         9

    accuracy                           0.36        11
   macro avg       0.61      0.61      0.36        11
weighted avg       0.86      0.36      0.36        11


Con 2.9009320735931396 segundos en ejecución.
Mejor random 

In [121]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.97441029548645 segundos en ejecución.
Mejor parámetro C: 2
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 13.539218187332153 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.22      1.00      0.36         2
Non-dystopia       1.00      0.22      0.36         9

    accuracy                           0.36        11
   macro avg       0.61      0.61      0.36        11
weighted avg       0.86      0.36      0.36        11


Con 2.8368661403656006 segundos en ejecución.
Mejor random s

In [122]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 7.24804162979126 segundos en ejecución.
Mejor parámetro C: 60
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.40      1.00      0.57         2
Non-dystopia       1.00      0.67      0.80         9

    accuracy                           0.73        11
   macro avg       0.70      0.83      0.69        11
weighted avg       0.89      0.73      0.76        11


Con 13.302724123001099 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.25      1.00      0.40         2
Non-dystopia       1.00      0.33      0.50         9

    accuracy                           0.45        11
   macro avg       0.62      0.67      0.45        11
weighted avg       0.86      0.45      0.48        11


Con 2.883535146713257 segundos en ejecución.
Mejor random 

In [123]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology', 'Characters, plot and writing style','Summary 1')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.699477195739746 segundos en ejecución.
Mejor parámetro C: 33
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 21.239441871643066 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.64078426361084 segundos en ejecución.
Mejor random 

In [124]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology', 'Characters, plot and writing style','Summary 2')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.466228723526001 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 21.570029497146606 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.723006248474121 segundos en ejecución.
Mejor random s

In [125]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology', 'Characters, plot and writing style','Summary 3')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.577098608016968 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 22.221923112869263 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.756904125213623 segundos en ejecución.
Mejor random s

In [126]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.813552618026733 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 18.45003390312195 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.564631462097168 segundos en ejecución.
Mejor random st

In [127]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','World view',  'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.3431220054626465 segundos en ejecución.
Mejor parámetro C: 3
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 16.44980025291443 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.5498311519622803 segundos en ejecución.
Mejor random 

In [128]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1', 'World view', 'Future projections','Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.315639972686768 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 16.9478120803833 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.6816022396087646 segundos en ejecución.
Mejor random state: 

In [129]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 1','World view', 'Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.605984449386597 segundos en ejecución.
Mejor parámetro C: 5
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 18.965748071670532 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.5158824920654297 segundos en ejecución.
Mejor random 

In [130]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.4384167194366455 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 18.600781202316284 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.5555970668792725 segundos en ejecución.
Mejor random

In [131]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2','World view',  'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.2672319412231445 segundos en ejecución.
Mejor parámetro C: 30
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 16.024102210998535 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.6498453617095947 segundos en ejecución.
Mejor rand

In [132]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'World view', 'Future projections','Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.296970844268799 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 16.94577980041504 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.597594738006592 segundos en ejecución.
Mejor random state: 

In [133]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2','World view', 'Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.502975940704346 segundos en ejecución.
Mejor parámetro C: 16
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 20.6467068195343 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.7621021270751953 segundos en ejecución.
Mejor random 

In [134]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.440279245376587 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 19.626796007156372 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.6815083026885986 segundos en ejecución.
Mejor random 

In [135]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2','World view',  'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.395112991333008 segundos en ejecución.
Mejor parámetro C: 30
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 15.912883520126343 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.7322964668273926 segundos en ejecución.
Mejor rando

In [136]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2', 'World view', 'Future projections','Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.387845277786255 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 17.335831880569458 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.676427125930786 segundos en ejecución.
Mejor random state:

In [137]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Summary 2','World view', 'Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 6.571137189865112 segundos en ejecución.
Mejor parámetro C: 16
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 19.7662832736969 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.670377731323242 segundos en ejecución.
Mejor random s

In [138]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.674063444137573 segundos en ejecución.
Mejor parámetro C: 27
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 12.708383321762085 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.6118698120117188 segundos en ejecución.
Mejor rando

In [139]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Future projections', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.427277565002441 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 9.816479444503784 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.3651301860809326 segundos en ejecución.
Mejor random s

In [140]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Ideology', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.411941051483154 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 9.601008653640747 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.3079307079315186 segundos en ejecución.
Mejor random s

In [141]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.456281900405884 segundos en ejecución.
Mejor parámetro C: 3
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 10.677701950073242 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.3155500888824463 segundos en ejecución.
Mejor random state

In [142]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.343693017959595 segundos en ejecución.
Mejor parámetro C: 8
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.67      1.00      0.80         2
Non-dystopia       1.00      0.89      0.94         9

    accuracy                           0.91        11
   macro avg       0.83      0.94      0.87        11
weighted avg       0.94      0.91      0.92        11


Con 10.449351787567139 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.397519826889038 segundos en ejecución.
Mejor random 

In [143]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Characters, plot and writing style','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.347102642059326 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 8.698991298675537 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.40      1.00      0.57         2
Non-dystopia       1.00      0.67      0.80         9

    accuracy                           0.73        11
   macro avg       0.70      0.83      0.69        11
weighted avg       0.89      0.73      0.76        11


Con 2.1869425773620605 segundos en ejecución.
Mejor random state: 

In [144]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Characters, plot and writing style','Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.309690713882446 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: sigmoid
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.25      0.50      0.33         2
Non-dystopia       0.86      0.67      0.75         9

    accuracy                           0.64        11
   macro avg       0.55      0.58      0.54        11
weighted avg       0.75      0.64      0.67        11


Con 8.303942680358887 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.20      0.50      0.29         2
Non-dystopia       0.83      0.56      0.67         9

    accuracy                           0.55        11
   macro avg       0.52      0.53      0.48        11
weighted avg       0.72      0.55      0.60        11


Con 2.3021016120910645 segundos en ejecución.
Mejor random 

In [145]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Characters, plot and writing style','Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.30400538444519 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 7.6318135261535645 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: liblinear
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 2.2351787090301514 segundos en ejecución.
Mejor random state: 

In [146]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.297081232070923 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 7.979698181152344 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.2320213317871094 segundos en ejecución.
Mejor random s

In [147]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view', 'Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.321905851364136 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 8.506055355072021 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.279677629470825 segundos en ejecución.
Mejor random st

In [148]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Future projections', 'Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.353886127471924 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 7.437190055847168 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 2.3039355278015137 segundos en ejecución.
Mejor random s

In [149]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','World view')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.217438459396362 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 6.72418737411499 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      1.00      0.67         2
Non-dystopia       1.00      0.78      0.88         9

    accuracy                           0.82        11
   macro avg       0.75      0.89      0.77        11
weighted avg       0.91      0.82      0.84        11


Con 2.1374447345733643 segundos en ejecución.
Mejor random st

In [150]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Future projections')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.200459241867065 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 6.520657539367676 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 2.169224500656128 segundos en ejecución.
Mejor random st

In [151]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Ideology')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.185386419296265 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.50      0.50      0.50         2
Non-dystopia       0.89      0.89      0.89         9

    accuracy                           0.82        11
   macro avg       0.69      0.69      0.69        11
weighted avg       0.82      0.82      0.82        11


Con 6.2195141315460205 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.33      0.50      0.40         2
Non-dystopia       0.88      0.78      0.82         9

    accuracy                           0.73        11
   macro avg       0.60      0.64      0.61        11
weighted avg       0.78      0.73      0.75        11


Con 2.172339916229248 segundos en ejecución.
Mejor random s

In [152]:
X_train, y_train, X_val, y_val = entrenarAlgoritmo('SciFiCorpus.csv','Characters, plot and writing style')
validacion(1)
print()
validacion(2)
print()
validacion(3)
print()
validacion(4)

Con 5.281450510025024 segundos en ejecución.
Mejor parámetro C: 1
Mejor kernel: linear
Reporte de clasificación: reporte_SVM.csv 
               precision    recall  f1-score   support

    Dystopia       0.00      0.00      0.00         2
Non-dystopia       0.75      0.67      0.71         9

    accuracy                           0.55        11
   macro avg       0.38      0.33      0.35        11
weighted avg       0.61      0.55      0.58        11


Con 6.745285749435425 segundos en ejecución.
Mejor max iteraciones: 800
Mejor solver: newton-cholesky
Reporte de clasificación: reporte_regression.csv 
               precision    recall  f1-score   support

    Dystopia       0.00      0.00      0.00         2
Non-dystopia       0.75      0.67      0.71         9

    accuracy                           0.55        11
   macro avg       0.38      0.33      0.35        11
weighted avg       0.61      0.55      0.58        11


Con 2.1440529823303223 segundos en ejecución.
Mejor random s